In [ ]:
start = time.time()
def time2Mins(time_):
    hr2min, mins = time_.split(':')
    return (int(hr2min)*60)+int(mins)

def date2Mins(date):
    date_format = "%m/%d/%Y"
    d0 = datetime.strptime(str(date), date_format)
    d1 = datetime.strptime('07/31/2019', date_format)
    delta = d0 - d1
    return int((delta.days)*24*60)


df = dd.read_csv('./data/csv/2769/2769_*.csv')
# df can fit in mem so use persist
df = df.persist()
df.datestamp = df.datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df.timestamp = df.timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df['time_anchor'] = df.datestamp + df.timestamp

lane_list = list(df['lane_type'].unique().compute())
lane_df_dict = {}
for idx, lane in enumerate(lane_list):
    lane_df_dict[idx] = df.loc[lambda df: df['lane_type'] == lane]
    lane_df_dict[idx].to_csv(f'./data/2769_{lane}.csv', single_file = True)
    
end = time.time()
f'the process took: {end-start}'

In [ ]:
start = time.time()
# join 37 files into a single df
df = dd.read_csv('./data/csv/2769/2769_*.csv')
df = df.repartition(npartitions=4)
# df can fit in mem so use persist
df = df.persist()
# map columns and create new column
df.datestamp = df.datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df.timestamp = df.timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df['time_anchor'] = df.datestamp + df.timestamp
df.visualize(filename='transpose.svg')



print(df)
# find the number of lanes of the road and split the data
lane_list = list(df['lane_type'].unique())
lane_df_dict = {}
for lane in lane_list:
    
    
    
    
    
    
    df.loc[lambda df: df['lane_type'] == lane].to_csv(f'./data/2769_{lane}.csv', single_file = True)
    
end = time.time()
f'the process took: {end-start}'

In [ ]:
start = time.time()
# join 37 files into a single df
df = dd.read_csv('./data/csv/2769/2769_*.csv')
df.set_index('lane_type')
# df.index = df.lane_type

df = df.repartition(npartitions=4)

df = df.map_partitions(lambda x:  x.sort_index())
# df can fit in mem so use persist

# df = df.persist()
# map columns and create new column
df.datestamp = df.datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df.timestamp = df.timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
df['time_anchor'] = df.datestamp + df.timestamp

df.to_csv('./data/*.csv')

end = time.time()
f'the process took: {end-start}'



# df.visualize(filename='transpose.svg')
# lane_list = list(df['lane_type'].unique())
# ddf_selected = df.map_partitions(lambda x: x[x['lane_type'] == lane_list[0]], meta = df.dtypes)

# print(df)
# # find the number of lanes of the road and split the data

# lane_df_dict = {}
# for lane in lane_list:

    
#     df.loc[lambda df: .to_csv(f'./data/2769_{lane}.csv', single_file = True)
    
# end = time.time()
# f'the process took: {end-start}'

In [ ]:
start = time.time()
# join 37 files into a single df
ddf = dd.read_csv('./data/csv/2769/2769_*.csv')
ddf = ddf.repartition(npartitions=3)
ddf = ddf.set_index('lane_type', sorted=True)
ddf = ddf.map_partitions(lambda x:  x.sort_index())
# map columns and create new column
ddf.datestamp = ddf.datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
ddf.timestamp = ddf.timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
ddf = ddf.map_partitions(lambda df: df.assign(time_anchor=(df.datestamp + df.timestamp)))
# df can fit in mem so use persist
ddf = ddf.persist()
end = time.time()
print(f'1st process took: {end-start}')
start = time.time()
# df.to_parquet('./data/temp.parq',write_index=False)
# df.to_feather('/data/testdf.feather')
ddf.to_csv('./data/*.csv')
end = time.time()
print(f'2nd process took: {end-start}')

In [ ]:
start = time.time()
# join 37 files into a single df
ddf = dd.read_csv('./data/csv/2769/2769_*.csv')
ddf = ddf.repartition(npartitions=4)
# ddf = ddf.set_index('lane_type', sorted=True)
# ddf = ddf.map_partitions(lambda x:  x.sort_index())
# map columns and create new column
ddf.datestamp = ddf.datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
ddf.timestamp = ddf.timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
ddf = ddf.map_partitions(lambda df: df.assign(z=(df.datestamp + df.timestamp)))
# df can fit in mem so use persist
ddf = ddf.persist()
end = time.time()
print(f'1st process took: {end-start}')
start = time.time()
# df.to_parquet('./data/temp.parq',write_index=False)
# df.to_feather('/data/testdf.feather')
ddf.to_csv('./data/*.csv')
# ddf.to_parquet('./data/iiii.csv', write_index=False)

end = time.time()
print(f'2nd process took: {end-start}')

In [ ]:
import os
start = time.time()
stations_dir = './data/batched/errored2'
dfs = {}
for station in os.listdir(stations_dir):
    file_path = f'{stations_dir}/{station}'
    # join 37 files into a single df
    dfs[station] = dd.read_csv(f'{file_path}/*.csv')
    dfs[station] = dfs[station].repartition(npartitions=4)
    dfs[station] = dfs[station].set_index('lane_type', sorted=True)
    dfs[station] = dfs[station].map_partitions(lambda x:  x.sort_index())
    # map columns and create new column
    dfs[station].datestamp = dfs[station].datestamp.map(lambda x: date2Mins(x), meta=pd.Series([], dtype=str, name='x'))
    dfs[station].timestamp = dfs[station].timestamp.map(lambda x: time2Mins(x), meta=pd.Series([], dtype=str, name='x'))
    dfs[station]['time_anchor'] = dfs[station].datestamp + dfs[station].timestamp
    # df can fit in mem so use persist
    dfs[station] = dfs[station].persist()
    end = time.time()
    print(f'1st process took: {end-start}')